In [17]:
import pandas as pd
import numpy as np
import sqlite3

from plotnine import *
from mizani.formatters import comma_format, percent_format
from datetime import datetime

start_date = "01/01/1960"
end_date = "12/31/2022"

In [18]:
import wrds
conn = wrds.Connection()

Loading library list...
Done


In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

connection_string = (
  "postgresql+psycopg2://"
 f"{os.getenv('WRDS_USER')}:{os.getenv('WRDS_PASSWORD')}"
  "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

wrds = create_engine(connection_string, pool_pre_ping=True)

In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

crsp_monthly_query = (
  "SELECT msf.permno, msf.date, "
         "date_trunc('month', msf.date)::date as month, "
         "msf.ret, msf.shrout, msf.altprc, "
         "msenames.exchcd, msenames.siccd, "
         "msedelist.dlret, msedelist.dlstcd "
    "FROM crsp.msf AS msf "
    "LEFT JOIN crsp.msenames as msenames "
    "ON msf.permno = msenames.permno AND "
       "msenames.namedt <= msf.date AND "
       "msf.date <= msenames.nameendt "
    "LEFT JOIN crsp.msedelist as msedelist "
    "ON msf.permno = msedelist.permno AND "
       "date_trunc('month', msf.date)::date = "
       "date_trunc('month', msedelist.dlstdt)::date "
   f"WHERE msf.date BETWEEN '{start_date}' AND '{end_date}' "
          "AND msenames.shrcd IN (10, 11)"
)

crsp_monthly = (pd.read_sql_query(
    sql=crsp_monthly_query,
    con=wrds,
    dtype={"permno": int, "exchcd": int, "siccd": int},
    parse_dates={"date", "month"})
  .assign(shrout=lambda x: x["shrout"]*1000)
)


/var/folders/_y/0xw7_0ss0s3f4kfhj9xl96zw0000gn/T/ipykernel_11569/2546435225.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.


AttributeError: module 'wrds' has no attribute 'cursor'